In [139]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset ,DataLoader
import itertools as it

### Input / output format
Input is in de vorm van twee vectors die waardes 1 en 0 bevatten (waar of onwaar).\
De waardes van de **eerste** vector moeten worden gelezen alsof er conjuncten tussen staan, een voorbeeld is de vector (0,1) die kan worden gelezen als: niet A en B.\
De waardes van de **tweede** vector moeten worden gelezen alsof er disjuncten tussen staan, verder zijn er altijd exact twee true values in de vector.\
Deze vectoren worden vergeleken (bijvoorbeeld (0,0,1) en (0,1,1) is True) en de output zal true of false zijn.\
De hoeveelheid mogelijke combinaties zijn 2^n * (n!/(2!(n-2)!)), voor n = 3 is dit 24.\
\
Alle combinaties waar de eerste vector gelijk is aan de tweede vector worden verwijderd.\
\
In deze poging ga ik de tien keer regel gebruiken, dit houd in dat voor elk mogelijk input-output paar er tien kopieëren.

In [140]:
#generate input vectors
def generate_input_vectors(n):
    fist_vec = list(it.product([0, 1], repeat=n))
    second_vec = []
    for i in fist_vec:
        if i.count(1) == 2:
            second_vec.append(i)
    total_vec = list(it.product(fist_vec, second_vec))
    for i in total_vec:
        if i[0] == i[1]:
            total_vec.remove(i)
    return total_vec
#generate true output
def generate_true(x):
    out = []
    for pair in x:
        for i in range(len(pair[0])):
            if pair[0][i] == 1 and pair[1][i] == 1:
                out.append(((pair[0],pair[1]),True))
                break
            elif i == len(pair[0]) - 1:
                out.append(((pair[0],pair[1]),False))
    return out
print(generate_true(generate_input_vectors(3)))

[(((0, 0, 0), (0, 1, 1)), False), (((0, 0, 0), (1, 0, 1)), False), (((0, 0, 0), (1, 1, 0)), False), (((0, 0, 1), (0, 1, 1)), True), (((0, 0, 1), (1, 0, 1)), True), (((0, 0, 1), (1, 1, 0)), False), (((0, 1, 0), (0, 1, 1)), True), (((0, 1, 0), (1, 0, 1)), False), (((0, 1, 0), (1, 1, 0)), True), (((0, 1, 1), (1, 0, 1)), True), (((0, 1, 1), (1, 1, 0)), True), (((1, 0, 0), (0, 1, 1)), False), (((1, 0, 0), (1, 0, 1)), True), (((1, 0, 0), (1, 1, 0)), True), (((1, 0, 1), (0, 1, 1)), True), (((1, 0, 1), (1, 1, 0)), True), (((1, 1, 0), (0, 1, 1)), True), (((1, 1, 0), (1, 0, 1)), True), (((1, 1, 1), (0, 1, 1)), True), (((1, 1, 1), (1, 0, 1)), True), (((1, 1, 1), (1, 1, 0)), True)]


In [141]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.X = [torch.tensor(i[0]) for i in data]
        self.y = torch.tensor([i[1] for i in data], dtype=torch.bool)
        self.data = list(zip(self.X, self.y))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [142]:
data = generate_true(generate_input_vectors(5))
dataset = CustomDataset(data)
dataloader_train = DataLoader(dataset, batch_size=4, shuffle=True)
dataloader_val = DataLoader(dataset, batch_size=4, shuffle=True)

In [143]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 2)

    def forward(self, x):
        x = x.view(-1, 10)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = NeuralNet()

In [144]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [145]:
epochs = 1000
for epoch in range(epochs):
    model.train()
    for inputs, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {loss.item()}')

Epoch: [1/1000], Loss: 0.9682886600494385
Epoch: [2/1000], Loss: 0.5890247821807861
Epoch: [3/1000], Loss: 0.3079911768436432
Epoch: [4/1000], Loss: 0.3853124678134918
Epoch: [5/1000], Loss: 0.5858852863311768
Epoch: [6/1000], Loss: 1.4553664922714233
Epoch: [7/1000], Loss: 0.21674329042434692
Epoch: [8/1000], Loss: 0.6210575103759766
Epoch: [9/1000], Loss: 0.22672627866268158
Epoch: [10/1000], Loss: 0.373146116733551
Epoch: [11/1000], Loss: 0.27023422718048096
Epoch: [12/1000], Loss: 0.5716463327407837
Epoch: [13/1000], Loss: 0.5531429648399353
Epoch: [14/1000], Loss: 0.3617790937423706
Epoch: [15/1000], Loss: 0.22530487179756165
Epoch: [16/1000], Loss: 0.21910415589809418
Epoch: [17/1000], Loss: 0.12658557295799255
Epoch: [18/1000], Loss: 0.3367706835269928
Epoch: [19/1000], Loss: 0.5948202013969421
Epoch: [20/1000], Loss: 0.37487369775772095
Epoch: [21/1000], Loss: 0.4346020519733429
Epoch: [22/1000], Loss: 1.2564401626586914
Epoch: [23/1000], Loss: 0.3031079173088074
Epoch: [24/100

In [146]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloader_val:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 87.74193548387096%


In [147]:
# inputs, labels = next(iter(dataloader_val))
# outputs = model(inputs.float())
# _, predicted = torch.max(outputs.data, 1)

# fig, ax = plt.subplots(1, 4, figsize=(12, 3))
# for i, axes in enumerate(ax):
#     axes.imshow(inputs[i].view(2, 5))
#     axes.set_title(f'Predicted: {predicted[i]}, Actual: {labels[i]}')
#     axes.axis('off')
# plt.show()